# Lab: Introduction to Notebooks for SPSS Professionals - Part 2
In part 2 of the lab you will learn how to load the model that was created in Part 1 and use it to score new data. 

### Step 1: Connect to Object Storage
If you wish, follow instructions in Appendix A of Lab Instructions to switch to your own object storage

In [ ]:
from pyspark.sql import SparkSession

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_18c4556616c5444581b1cb6d212cf2dc(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', '879160f1a1174d2f912f196ac158ffbf')
    hconf.set(prefix + '.username', 'aa5ea4cb9c48463681897f88b4a9ab08')
    hconf.set(prefix + '.password', 'veCB_4)bYn362UU&')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_18c4556616c5444581b1cb6d212cf2dc(name)

spark = SparkSession.builder.getOrCreate()

### Step 2: Load the new data for scoring
Note: The test data set contains all fields that were used for model training with the exception of the target variable (MortgageDefault)

In [ ]:
newData = spark.read.format('csv')\
  .options(header='true', inferschema='true')\
  .load("swift://IntroToNotebooks." + name + "/MortgageDefaultTestData.csv")

newData.toPandas().head()

### Step 3: Load Saved Model
The model was saved in the model building notebook. If you changed the namespace (in our example ConvertSPSSModelToNotebook) in that notebook, modify it to the changed value here. 

In [ ]:
#Load Saved Model
from pyspark.ml import PipelineModel
model = PipelineModel.load("IntroToNotebooks.mortgageDefaultModel")

### Step 4: Score Reloaded Model
Take note of the prediction and probability columns in the result

In [ ]:
#Score reloaded model
# Take note of the prediction and probability columns in the result

results = model.transform(newData)
results.toPandas().head(5)

### Step 5: Write results into a csv file

In [ ]:
#Select ID, prediction and probability fields from the results dataframe
r1=results.select(results["ID"], results["prediction"],results["probability"])
r1.show(5,False)

#### Decompose the probability column
The probability column contains a vector for each record, and the elements must be extracted

In [ ]:
from pyspark.sql import Row
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors

udf_0 = udf(lambda vector: float(vector[0]), DoubleType())
udf_1 = udf(lambda vector: float(vector[1]), DoubleType())

r2 = (r1.select(r1["ID"], r1["prediction"],r1["probability"])
    .withColumn('probability_0', udf_0(r1.probability))
    .withColumn('probability_1', udf_1(r1.probability))
    .drop("probability"))

r2.show(5, False)

In [ ]:
r2.write.csv('swift://IntroToNotebooks.' + name + '/mortgage_default_scores.csv', mode='overwrite')

In [ ]:
# Show csv file can be read back
r3= spark.read.csv('swift://IntroToNotebooks.' + name + '/mortgage_default_scores.csv')
r3.select(r3["_c0"].alias("ID"), r3["_c1"].alias("prediction"), r3["_c2"].alias("probability_0"), r3["_c3"].alias("probability_1")).show(5, False)

You have finished testing of the scoring notebook

### Step 6: Set up a schedule to run a notebook
Use the clock icon in the top right corner to schedule the notebook. 